In [96]:
# Importing Libraries
import numpy as np
import pandas as pd

In [97]:
from helpers import BotScrap

## Here I will trying to filter data from FBREF

In [98]:
data = BotScrap()
match = data.get_match('https://fbref.com/en/matches/abf3df21/South-Africa-Italy-August-2-2023-FIFA-Womens-World-Cup')

In [102]:
home = match['Away Player Stats'][0].values[0][0]
print(type(home))
home

<class 'pandas.core.frame.DataFrame'>


,Italy (4-2-3-1),Italy (4-2-3-1).1
0,22,Francesca Durante
1,3,Benedetta Orsi
2,4,Lucia Di Guglielmo
3,5,Elena Linari
4,6,Manuela Giugliano
5,8,Barbara Bonansea
6,9,Valentina Giacinti
7,14,Chiara Beccari
8,16,Giulia Dragoni
9,17,Lisa Boattin
